<a href="https://colab.research.google.com/github/Est17256/ONNX-Pruebas/blob/main/Onnx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Configuracion inicial

In [ ]:
###############################Coneccion a Spark###############################
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.2/spark-3.0.2-bin-hadoop2.7.tgz
!tar xf spark-3.0.2-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.2-bin-hadoop2.7"
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
###############################################################################

In [ ]:
###############################Importacion de librerias externas###############################
%pip install onnxmltools
%pip install onnxruntime
###############################################################################################

     |████████████████████████████████| 256kB 8.9MB/s 
     |████████████████████████████████| 102kB 7.3MB/s 
     |████████████████████████████████| 71kB 6.9MB/s 
     |████████████████████████████████| 14.5MB 328kB/s 
     |████████████████████████████████| 235kB 44.9MB/s 
     |████████████████████████████████| 92kB 8.6MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115928 sha256=ded97373d9eb6154e4ef481f69800735304f56c95c40fe3a8f9b430d06aff8e9
  Stored in directory: /root/.cache/pip/wheels/af/19/30/1ea0cad502dcb4e66ed5a690279628c827aea38bbbab75d5ed
Successfully built fire
     |████████████████████████████████| 4.1MB 8.7MB/s 


In [ ]:
###############################Importacion de librerias###############################
from pyspark.ml.feature import StringIndexer,VectorAssembler
from pyspark.ml.classification import NaiveBayes, LogisticRegression, DecisionTreeClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from onnxmltools import convert_sparkml
from onnxmltools.convert.sparkml.utils import buildInitialTypesSimple
import numpy as np
import onnxruntime 
######################################################################################

## Preparacion de Data

Se carga la data

In [ ]:
df =spark.read.csv("/content/iris.csv",header=True,inferSchema=True)
df.show()

+------------+-----------+------------+-----------+-------+
|sepal_length|sepal_width|petal_length|petal_width|species|
+------------+-----------+------------+-----------+-------+
|         5.1|        3.5|         1.4|        0.2| setosa|
|         4.9|        3.0|         1.4|        0.2| setosa|
|         4.7|        3.2|         1.3|        0.2| setosa|
|         4.6|        3.1|         1.5|        0.2| setosa|
|         5.0|        3.6|         1.4|        0.2| setosa|
|         5.4|        3.9|         1.7|        0.4| setosa|
|         4.6|        3.4|         1.4|        0.3| setosa|
|         5.0|        3.4|         1.5|        0.2| setosa|
|         4.4|        2.9|         1.4|        0.2| setosa|
|         4.9|        3.1|         1.5|        0.1| setosa|
|         5.4|        3.7|         1.5|        0.2| setosa|
|         4.8|        3.4|         1.6|        0.2| setosa|
|         4.8|        3.0|         1.4|        0.1| setosa|
|         4.3|        3.0|         1.1| 

In [ ]:
df.printSchema()

root
 |-- sepal_length: double (nullable = true)
 |-- sepal_width: double (nullable = true)
 |-- petal_length: double (nullable = true)
 |-- petal_width: double (nullable = true)
 |-- species: string (nullable = true)



Se realiza el string indexer de la variable a predecir

In [ ]:
labelIndexer = StringIndexer(inputCol="species", outputCol="label")
trainingData2=labelIndexer.fit(df)
df=trainingData2.transform(df)

In [ ]:
df.show(5)

+------------+-----------+------------+-----------+-------+-----+
|sepal_length|sepal_width|petal_length|petal_width|species|label|
+------------+-----------+------------+-----------+-------+-----+
|         5.1|        3.5|         1.4|        0.2| setosa|  0.0|
|         4.9|        3.0|         1.4|        0.2| setosa|  0.0|
|         4.7|        3.2|         1.3|        0.2| setosa|  0.0|
|         4.6|        3.1|         1.5|        0.2| setosa|  0.0|
|         5.0|        3.6|         1.4|        0.2| setosa|  0.0|
+------------+-----------+------------+-----------+-------+-----+
only showing top 5 rows



Se elimina el texto de la variable a predecir

In [ ]:
df=df.drop("species")

In [ ]:
df.show()

+------------+-----------+------------+-----------+-----+
|sepal_length|sepal_width|petal_length|petal_width|label|
+------------+-----------+------------+-----------+-----+
|         5.1|        3.5|         1.4|        0.2|  0.0|
|         4.9|        3.0|         1.4|        0.2|  0.0|
|         4.7|        3.2|         1.3|        0.2|  0.0|
|         4.6|        3.1|         1.5|        0.2|  0.0|
|         5.0|        3.6|         1.4|        0.2|  0.0|
|         5.4|        3.9|         1.7|        0.4|  0.0|
|         4.6|        3.4|         1.4|        0.3|  0.0|
|         5.0|        3.4|         1.5|        0.2|  0.0|
|         4.4|        2.9|         1.4|        0.2|  0.0|
|         4.9|        3.1|         1.5|        0.1|  0.0|
|         5.4|        3.7|         1.5|        0.2|  0.0|
|         4.8|        3.4|         1.6|        0.2|  0.0|
|         4.8|        3.0|         1.4|        0.1|  0.0|
|         4.3|        3.0|         1.1|        0.1|  0.0|
|         5.8|

Se crea la division de datos para trabajar

In [ ]:
(trainingData, testData) = df.randomSplit([0.7, 0.3], seed = 100)

In [ ]:
datapruebas=testData.drop("label")
datapruebas=datapruebas.toPandas()
datapruebas

,sepal_length,sepal_width,petal_length,petal_width
0,4.4,3.2,1.3,0.2
1,4.7,3.2,1.6,0.2
2,4.8,3.0,1.4,0.1
3,4.8,3.4,1.6,0.2
4,4.8,3.4,1.9,0.2
5,4.9,3.1,1.5,0.1
6,4.9,3.1,1.5,0.1
7,5.0,3.2,1.2,0.2
8,5.0,3.5,1.3,0.3
9,5.1,3.4,1.5,0.2


Se crea el vectorAssembler para unir las variables con atributos

In [ ]:
vecAssembler = VectorAssembler(inputCols=["sepal_length", "sepal_width", "petal_length", "petal_width"], outputCol="features")

## NaiveBayes

In [ ]:
#Se asignan valores al modelo
nb = NaiveBayes(featuresCol = 'features', labelCol = 'label',smoothing=1.0, modelType="multinomial")
#Se unen en la pipeline
pipeline = Pipeline(stages=[vecAssembler, nb])
#Se entrena el modelo
modelNB = pipeline.fit(trainingData)
#Se realiza la prediccion con los datos de test
predictions = modelNB.transform(testData)
#Se muestran los resultados
predictions.select("label", "prediction").show()

+-----+----------+
|label|prediction|
+-----+----------+
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  1.0|       1.0|
+-----+----------+
only showing top 20 rows



In [ ]:
#Se verifica el porcentaje de efectividad
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

0.9126293995859213

## LogisticRegression

In [ ]:
#Se asignan valores al modelo
lr = LogisticRegression(featuresCol = 'features', labelCol = 'label',maxIter=10, regParam=0.3, elasticNetParam=0)
#Se unen en la pipeline
pipeline = Pipeline(stages=[vecAssembler, lr])
#Se entrena el modelo
modelLR = pipeline.fit(trainingData)
#Se realiza la prediccion con los datos de test
predictions = modelLR.transform(testData)
#Se muestran los resultados
predictions.select("label", "prediction").show()

+-----+----------+
|label|prediction|
+-----+----------+
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       2.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  1.0|       1.0|
+-----+----------+
only showing top 20 rows



In [ ]:
#Se verifica el porcentaje de efectividad
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

0.9333243316230083

## Decision Tree Classifier

In [ ]:
#Se asignan valores al modelo
dtc = DecisionTreeClassifier(labelCol="label", featuresCol="features")
#Se unen en la pipeline
pipeline = Pipeline(stages=[vecAssembler, dtc])
#Se entrena el modelo
modelDTC = pipeline.fit(trainingData)
#Se realiza la prediccion con los datos de test
predictions = modelDTC.transform(testData)
#Se muestran los resultados
predictions.select("label", "prediction").show()

+-----+----------+
|label|prediction|
+-----+----------+
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  1.0|       1.0|
+-----+----------+
only showing top 20 rows



In [ ]:
#Se verifica el porcentaje de efectividad
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

0.9344664031620552

## Decision Tree Classifier a ONNX

In [ ]:
sample_input = testData.drop("species","label")
initial_types = buildInitialTypesSimple(sample_input) 
onnx_model = convert_sparkml(modelDTC, 'My Sparkml Pipeline', initial_types, spark_session = spark)

The maximum opset needed by this model is only 4.


In [ ]:
with open("/content/DTC.onnx", "wb") as f:
  f.write(onnx_model.SerializeToString())

In [ ]:
session = onnxruntime.InferenceSession("/content/DTC.onnx", None)
output = session.get_outputs()[0]
inputs = session.get_inputs()
x1=0
x2=4
for i in range(45):
  x1+=4
  x2+=4
  # print(x1,x2)
  results = session.run([output.name], {i.name: v for i, v in zip(inputs,datapruebas.values.reshape(184,1,1).astype(np.float32)[x1:x2] )})
  print(results[0][0])

0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
0
1
1
2
1
1
0
2
2
1
1
1
1
2
2
2
1
2
2
2
2
2
2
2
1
2
2
2
2


## NaiveBayes a ONNX

In [ ]:
sample_input = testData.drop("species","label")
initial_types = buildInitialTypesSimple(sample_input) 
onnx_model = convert_sparkml(modelNB, 'My Sparkml Pipeline', initial_types, spark_session = spark)

The maximum opset needed by this model is only 9.
The maximum opset needed by this model is only 4.


In [ ]:
with open("/content/NB.onnx", "wb") as f:
  f.write(onnx_model.SerializeToString())

In [ ]:
session = onnxruntime.InferenceSession("/content/NB.onnx", None)
output = session.get_outputs()[0]
inputs = session.get_inputs()
x1=0
x2=4
for i in range(45):
  x1+=4
  x2+=4
  # print(x1,x2)
  results = session.run([output.name], {i.name: v for i, v in zip(inputs,datapruebas.values.reshape(184,1,1).astype(np.float32)[x1:x2] )})
  print(results[0][0])

InvalidGraph: ignored

## RegresionLogistica a ONNX

In [ ]:
sample_input = testData.drop("species","label")
initial_types = buildInitialTypesSimple(sample_input) 
onnx_model = convert_sparkml(modelLR, 'My Sparkml Pipeline', initial_types, spark_session = spark)

The maximum opset needed by this model is only 4.


{'classlabels_ints': [0, 1, 2],
 'coefficients': [-0.2584586831740849,
                  0.9104817675498127,
                  -0.2394833636441817,
                  -0.5368312048218311,
                  0.2163059194512747,
                  -0.40680933186708396,
                  0.050113589435907734,
                  -0.048910095720332045,
                  0.04215276372280986,
                  -0.5036724356827297,
                  0.18936977420827372,
                  0.5857413005421629],
 'intercepts': [-0.0098275151331854, 0.1230454302571317, -0.11321791512394629],
 'multi_class': 1,
 'name': 'LinearClassifier',
 'post_transform': 'LOGISTIC'}


In [ ]:
with open("/content/LR.onnx", "wb") as f:
  f.write(onnx_model.SerializeToString())

In [ ]:
session = onnxruntime.InferenceSession("/content/LR.onnx", None)
output = session.get_outputs()[0]
inputs = session.get_inputs()
x1=0
x2=4
for i in range(45):
  x1+=4
  x2+=4
  # print(x1,x2)
  results = session.run([output.name], {i.name: v for i, v in zip(inputs,datapruebas.values.reshape(184,1,1).astype(np.float32)[x1:x2] )})
  print(results[0][0])

0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
1
0
1
1
2
1
1
0
2
2
2
1
1
2
2
2
2
1
2
2
2
2
2
1
2
1
2
2
2
2
